In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_experiments.framework import ExperimentData
from qiskit_experiments.library.tomography import MitigatedTomographyAnalysis, MitigatedStateTomography, TomographyAnalysis
from qiskit_experiments.library.tomography.basis import PauliMeasurementBasis
from qiskit_experiments.library.characterization.analysis import LocalReadoutErrorAnalysis
from qiskit_ibm_runtime import QiskitRuntimeService

import scipy
from numpy import set_printoptions, radians, ndarray, pi, degrees, radians
from random import randint
from collections import namedtuple

import itertools
from itertools import product

In [2]:
# Set the floating point diplay precision to 2 decimal places, sufficient for our purposes.
set_printoptions(precision=3)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [3]:
states = [
    (radians(-104), radians(-146)),
    (radians(-158), radians(-108)),
    (radians(-110), radians(-172))
]

In [4]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(7)
    
    # Route on first layer
    circ.h(3)
    circ.u(theta, phi, 0, 2)
    circ.cswap(3, 2, 4)
    
    # Route on second layer, top
    circ.h(0)
    circ.cswap(0, 1, 2)
    
    # Route on second layer, bottom
    circ.h(6)
    circ.cswap(6, 4, 5)
    
    return circ

In [5]:
build_circuit(*states[0]).draw()

┌───┐                
q_0: ─────────┤ H ├──────────────■─
              └───┘              │ 
q_1: ────────────────────────────X─
     ┌──────────────────────┐    │ 
q_2: ┤ U(-1.8151,-2.5482,0) ├─X──X─
     └────────┬───┬─────────┘ │    
q_3: ─────────┤ H ├───────────■────
              └───┘           │    
q_4: ─────────────────────────X──X─
                                 │ 
q_5: ────────────────────────────X─
              ┌───┐              │ 
q_6: ─────────┤ H ├──────────────■─
              └───┘

In [8]:
def build_tomography_set(circuit: QuantumCircuit) -> QuantumCircuit:
    signal_paths = [1, 2, 4, 5]
    
    circuit_x = circuit.copy()
    circuit_x.h(signal_paths)
    circuit_x.measure_all()

    circuit_y = circuit.copy()
    circuit_y.sdg(signal_paths)
    circuit_y.h(signal_paths)
    circuit_y.measure_all()

    circuit_z = circuit.copy()
    circuit_z.measure_all()

    return [circuit_x, circuit_y, circuit_z]


In [9]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [10]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            build_tomography_set(build_circuit(theta, phi)) for theta, phi in states
        ]
    ]
))

circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [12]:
def get_all_physical_qubits(circuit_list: list[QuantumCircuit]) -> set[int]:
    all_indices = set()
    for circ in circuit_list:    
        measurement_indices = [instr[1][0]._index for instr in circ.data if instr[0].name == 'measure']
        all_indices = all_indices.union(measurement_indices)
    return all_indices


def get_mitigation_circuits(circuit_list: list[QuantumCircuit]) -> list[QuantumCircuit]:

    physical_qubits = get_all_physical_qubits(circuit_list)

    num_qubits = len(physical_qubits)
    all_0 = QuantumCircuit(127, num_qubits)
    all_0.measure(physical_qubits, range(num_qubits))

    all_1 = QuantumCircuit(127, num_qubits)
    all_1.x(physical_qubits)
    all_1.measure(physical_qubits, range(num_qubits))

    return [all_0, all_1]

In [13]:
circuits_to_send = get_mitigation_circuits(circuits_to_send) + circuits_to_send

In [14]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [15]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cw5pyfhvwdtg0081n9b0


---

In [63]:
result = hardware_job.result().get_counts()

### Calculate Unmitigated Fidelities

In [92]:
def get_signal_qubit_idx_from_measurement_outcome(bit_string: str) -> int:
    if bit_string[3] == '0':
        if bit_string[6] == '0':
            return 4
        return 5
    else:
        if bit_string[0] == '0':
            return 2
        return 1


def get_xyz_counts_for_circuit(counts_list: list) -> tuple[dict]:
    output = []
    for counts in counts_list:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[get_signal_qubit_idx_from_measurement_outcome(bit_string)] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[get_signal_qubit_idx_from_measurement_outcome(bit_string)] == '1')
        
        output.append(counts_dict)
    
    return tuple(output)


def print_unmitigated_fidelities_combined(counts, theta, phi):

    qubit = density_op_from_counts_dict(*get_xyz_counts_for_circuit(counts))

    print(get_xyz_counts_for_circuit(counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'State fidelity from combined counts: {fidelity_q}\n')

In [93]:
print_unmitigated_fidelities_combined(result[2:5], *states[0])
print_unmitigated_fidelities_combined(result[5:8], *states[1])
print_unmitigated_fidelities_combined(result[8:11], *states[2])

({'0': 3227, '1': 773}, {'0': 1741, '1': 2259}, {'0': 2433, '1': 1567})
[[0.608+0.j    0.307+0.065j]
 [0.307-0.065j 0.392+0.j   ]]
State fidelity from combined counts: 0.6854330013996655

({'0': 1924, '1': 2076}, {'0': 2244, '1': 1756}, {'0': 2732, '1': 1268})
[[ 0.683+0.j    -0.019-0.061j]
 [-0.019+0.061j  0.317+0.j   ]]
State fidelity from combined counts: 0.3498585128910752

({'0': 1813, '1': 2187}, {'0': 1100, '1': 2900}, {'0': 2299, '1': 1701})
[[ 0.575+0.j    -0.047+0.225j]
 [-0.047-0.225j  0.425+0.j   ]]
State fidelity from combined counts: 0.4015054085397613



### Calculate Mitigation Fidelities

In [42]:
# Gets the map from a physical qubit to a classical bit for the mitigation calibration matrices
def get_qubit_to_clbit_mappings_for_mitigation(circuit: QuantumCircuit) -> dict[int, int]:
    return dict(
        ((instr[1][0]._index, instr[2][0]._index) for instr in circuit.data if instr[0].name == 'measure')
    )

# Gets the map from physical qubit to mitigation matrix
def get_assignment_matrices(mappings: dict, counts_0: dict, counts_1: dict) -> dict[int, ndarray]:

    output = {}

    for physical_qubit, classical_bit in mappings.items():
        
        # Determine zero state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        zero_ket = array([[result_0], 
                          [result_1]]) / (result_0 + result_1)
        
        # Determine one state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        one_ket = array([[result_0], 
                         [result_1]]) / (result_0 + result_1)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        output[physical_qubit] = assignment_matrix
    
    return output

# Get the list of physical qubits in the same order as the classical bits 
def get_qubits_in_clbit_order(circuit: QuantumCircuit) -> list[int]:
    
    cl_bit_to_qbit_map = dict((instr[2][0]._index, instr[1][0]._index) for instr in circuit.data if instr[0].name == 'measure')

    return [*cl_bit_to_qbit_map.values()]
    

# Get the tensor of all mitigation matrices in the correct order for each circuit
def get_mitigation_matrix(circuit: QuantumCircuit, assignment_mappings: dict) -> ndarray:
    ordered_physical_qubits = get_qubits_in_clbit_order(circuit)

    ordered_physical_qubits.reverse()

    assignment_matrices = [assignment_mappings[qubit] for qubit in ordered_physical_qubits]

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def get_corrected_counts(circuit: QuantumCircuit, counts_for_mitigation: dict, assignment_mappings: dict, num_qubits: int) -> dict[str, int]:
    bit_strings = get_bit_strings(num_qubits)

    counts_list = []
    for bit_string in bit_strings:
        counts_list.append(counts_for_mitigation[bit_string] if bit_string in counts_for_mitigation else 0)
    
    # Get vector of counts
    counts_vector = numpy.concatenate(
        array([[count for count in counts_list]]),
        axis=0
    )

    mitigation_matrix = get_mitigation_matrix(circuit, assignment_mappings)
    
    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)
    
    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])
    
    return output

In [47]:
# Get list of qubit to clbit mappings for the mitigation circuits.
mitigation_mappings = get_qubit_to_clbit_mappings_for_mitigation(circuits_to_send[0])

# Calculate assignment matrices for each qubit. 
assignment_matrices = get_assignment_matrices(mitigation_mappings, result[0], result[1])

In [98]:
def print_mitigated_fidelities_combined(circuits, counts, assignment_matrices, num_qubits, theta, phi):

    mitigated_counts = []

    for circuit, count in zip(circuits, counts):
        mitigated_counts.append(get_corrected_counts(circuit, count, assignment_matrices, num_qubits))

    qubit = density_op_from_counts_dict(*get_xyz_counts_for_circuit(mitigated_counts))
    
    print(get_xyz_counts_for_circuit(mitigated_counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'Mitigated state fidelity from combined counts: {fidelity_q}\n')

In [99]:
print_mitigated_fidelities_combined(circuits_to_send[2:5], result[2:5], assignment_matrices, 7, *states[0])
print_mitigated_fidelities_combined(circuits_to_send[5:8], result[5:8], assignment_matrices, 7, *states[1])
print_mitigated_fidelities_combined(circuits_to_send[8:11], result[8:11], assignment_matrices, 7, *states[2])

({'0': 3261, '1': 674}, {'0': 1703, '1': 2234}, {'0': 2430, '1': 1506})
[[0.617+0.j    0.329+0.067j]
 [0.329-0.067j 0.383+0.j   ]]
Mitigated state fidelity from combined counts: 0.6994369589622998

({'0': 1858, '1': 2074}, {'0': 2213, '1': 1724}, {'0': 2717, '1': 1224})
[[ 0.689+0.j    -0.027-0.062j]
 [-0.027+0.062j  0.311+0.j   ]]
Mitigated state fidelity from combined counts: 0.34331986660772573

({'0': 1781, '1': 2150}, {'0': 1022, '1': 2916}, {'0': 2290, '1': 1647})
[[ 0.582+0.j   -0.047+0.24j]
 [-0.047-0.24j  0.418+0.j  ]]
Mitigated state fidelity from combined counts: 0.39694571835263903



In [67]:
for string, count in get_corrected_counts(circuits_to_send[2], result[2], assignment_matrices, 7).items():
    if not count == result[2][string]:
        print(f'{string} unmitigated: {result[2][string]} mitigated: {count} difference {count - result[2][string]}')

0000001 unmitigated: 46 mitigated: 47 difference 1
0000010 unmitigated: 34 mitigated: 33 difference -1
0000011 unmitigated: 6 mitigated: 4 difference -2
0000100 unmitigated: 14 mitigated: 13 difference -1
0000101 unmitigated: 57 mitigated: 59 difference 2
0000110 unmitigated: 7 mitigated: 6 difference -1
0000111 unmitigated: 6 mitigated: 4 difference -2
0001001 unmitigated: 57 mitigated: 56 difference -1
0001010 unmitigated: 54 mitigated: 52 difference -2
0001100 unmitigated: 43 mitigated: 42 difference -1
0001101 unmitigated: 64 mitigated: 63 difference -1
0001111 unmitigated: 36 mitigated: 35 difference -1
0010000 unmitigated: 45 mitigated: 47 difference 2
0010001 unmitigated: 46 mitigated: 48 difference 2
0010010 unmitigated: 56 mitigated: 59 difference 3
0010011 unmitigated: 5 mitigated: 3 difference -2
0010100 unmitigated: 10 mitigated: 9 difference -1
0010101 unmitigated: 51 mitigated: 54 difference 3
0010111 unmitigated: 10 mitigated: 9 difference -1
0011000 unmitigated: 5 mitig